In [0]:
# Import required libraries

import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [2]:
#Load MNIST dataset

mnist_ds = tf.keras.datasets.mnist

(xtrain, ytrain), (xtest, ytest) = mnist_ds.load_data()
xtrain, xtest = xtrain / 255.0, xtest / 255.0

# Add dimension of channel
xtrain = xtrain[..., tf.newaxis]
xtest = xtest[..., tf.newaxis]

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
# Shuffle the dataset using tf.data

train = tf.data.Dataset.from_tensor_slices(
    (xtrain, ytrain)).shuffle(10000).batch(32)

test = tf.data.Dataset.from_tensor_slices((xtest, ytest)).batch(32)


In [0]:
# Build the keras model

class My_tf_Model(Model):
  def __init__(self):
    super(My_tf_Model, self).__init__()
    self.conv = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10)

  def call(self, a):
    a = self.conv(a)
    a = self.flatten(a)
    a = self.d1(a)
    return self.d2(a)

# Create an instance of the model
model = My_tf_Model()

Otimizer and loss function for training

In [0]:
lossObject = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

Evaluate the model

In [0]:
trainLoss = tf.keras.metrics.Mean(name='trainLoss')
trainAccuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='trainAccuracy')

testLoss = tf.keras.metrics.Mean(name='testLoss')
testAccuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='testAccuracy')

Train the model using tf.gradient tape

In [0]:
@tf.function
def trainStep(image, label):
  with tf.GradientTape() as tape:
    pred = model(image, training=True)
    loss = lossObject(label, pred)
  gradient = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradient, model.trainable_variables))

  trainLoss(loss)
  trainAccuracy(label, pred)

Test the model

In [0]:
@tf.function
def testStep(image, label):

  pred = model(image, training=False)
  tLoss = lossObject(label, pred)

  testLoss(tLoss)
  testAccuracy(label, pred)

In [11]:
epoch = 10

for epoch in range(epoch):
  # Reset the metrics at the start of the next epoch
  trainLoss.reset_states()
  trainAccuracy.reset_states()
  testLoss.reset_states()
  testAccuracy.reset_states()

  for image, label in train:
    trainStep(image, label)

  for testImage, testLabel in test:
    testStep(testImage, testLabel)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch + 1,
                        trainLoss.result(),
                        trainAccuracy.result() * 100,
                        testLoss.result(),
                        testAccuracy.result() * 100))

Epoch 1, Loss: 0.043748099356889725, Accuracy: 98.66166687011719, Test Loss: 0.053835853934288025, Test Accuracy: 98.22000122070312
Epoch 2, Loss: 0.022416623309254646, Accuracy: 99.26499938964844, Test Loss: 0.0506158173084259, Test Accuracy: 98.33999633789062
Epoch 3, Loss: 0.012980688363313675, Accuracy: 99.5633316040039, Test Loss: 0.0650545284152031, Test Accuracy: 98.13999938964844
Epoch 4, Loss: 0.009142785333096981, Accuracy: 99.6883316040039, Test Loss: 0.06806790083646774, Test Accuracy: 98.37999725341797
Epoch 5, Loss: 0.0069343335926532745, Accuracy: 99.76166534423828, Test Loss: 0.06270670890808105, Test Accuracy: 98.4000015258789
Epoch 6, Loss: 0.005146103445440531, Accuracy: 99.83999633789062, Test Loss: 0.0748978853225708, Test Accuracy: 98.19999694824219
Epoch 7, Loss: 0.004850856028497219, Accuracy: 99.81999969482422, Test Loss: 0.06857401132583618, Test Accuracy: 98.36000061035156
Epoch 8, Loss: 0.0025050928816199303, Accuracy: 99.93499755859375, Test Loss: 0.1454413